In [1]:
using DataFrames
using POMDPs
using POMDPToolbox
using MCTS

## State Representation

In [2]:
# State struct definition
struct ReservoirState 
    drilled::Array{Int64} # List (of length 0 to 2) of previously drilled grid indices
    grid::Array{Int64}    # Whether each grid has oil underneath or not. 
                          # Also directly used as reward value. 
end

## POMDP Object Definition

In [3]:
# Definition of POMDP
# State: ReservoirState (<OilGridValues, DrillingHistory> pair)
# Action: Int64 (2d coordinate flattened to linear index)
# Observation: Array{Int64} (Grid values, each indicating HasRock = 0 or 1)
type ReservoirPOMDP <: POMDP{ReservoirState, Int64, Array{Int64}} 
    size_grid::Int64 # Size of the grid space (2d coordinate flattened to linear index) 
    penalty::Int64   # Penalty for drilling on a previously grid
    p_rock::Float64  # Probability that whether a grid has rock matches whether the grid has oil underneath
                     # p_rock == P( rock[ind]==0 | oil[ind]==0 ) == P( rock[ind]==1 | oil[ind]==1 )
end

In [4]:
# Default Constructor
# size_grid: 9 (3x3)
# penalty: -100
# p_rock: 0.8
function ReservoirPOMDP()
    return ReservoirPOMDP(9, -100, 0.8)
end

ReservoirPOMDP(9, -100, 0.8)

## State/Action/Observation Space Definition

In [5]:
# State space definition
println("Generating state space.")

# List of possible states
state_space = ReservoirState[]

# Generate states at depth 0
println("Generating states at depth 0...")
for g1 = 0:1, g2 = 0:1, g3 = 0:1, g4 = 0:1, g5 = 0:1, g6 = 0:1, g7 = 0:1, g8 = 0:1, g9 = 0:1
    grid = [g1, g2, g3, g4, g5, g6, g7, g8, g9]
    push!(state_space, ReservoirState([], grid))
end
println("Depth 0 done.")

# Generate states at depth 1
println("Generating states at depth 1...")
for i=1:pomdp.size_grid
    for g1 = 0:1, g2 = 0:1, g3 = 0:1, g4 = 0:1, g5 = 0:1, g6 = 0:1, g7 = 0:1, g8 = 0:1, g9 = 0:1
        grid = [g1, g2, g3, g4, g5, g6, g7, g8, g9]
        push!(state_space, ReservoirState([i], grid))
    end
end
println("Depth 1 done.")

# Generate states at depth 2
println("Generating states at depth 2...")
for i=1:pomdp.size_grid, j=1:pomdp.size_grid
    if (i != j) # Avoid duplicate drilling
        for g1 = 0:1, g2 = 0:1, g3 = 0:1, g4 = 0:1, g5 = 0:1, g6 = 0:1, g7 = 0:1, g8 = 0:1, g9 = 0:1
            grid = [g1, g2, g3, g4, g5, g6, g7, g8, g9]
            push!(state_space, ReservoirState([i, j], grid))
        end
    end
end
println("Depth 2 done.")

# Generate states at depth 3
println("Generating states at depth 3...")
for i=1:pomdp.size_grid, j=1:pomdp.size_grid, k=1:pomdp.size_grid
    if (!((i == j) || (j == k) || (k == i)))
        for g1 = 0:1, g2 = 0:1, g3 = 0:1, g4 = 0:1, g5 = 0:1, g6 = 0:1, g7 = 0:1, g8 = 0:1, g9 = 0:1
            grid = [g1, g2, g3, g4, g5, g6, g7, g8, g9]
            push!(state_space, ReservoirState([i, j, k], grid))
        end
    end
end
println("Depth 3 done.")

println("State space generated.")
println("Length of state_space: ", length(state_space))

POMDPs.states(pomdp::ReservoirPOMDP) = state_space

# Test
POMDPs.states(pomdp)

Generating state space.
Generating states at depth 0...
Depth 0 done.
Generating states at depth 1...
Depth 1 done.
Generating states at depth 2...
Depth 2 done.
Generating states at depth 3...
Depth 3 done.
State space generated.
Length of state_space: 300032


300032-element Array{ReservoirState,1}:
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 0, 0, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 0, 0, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 0, 1, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 0, 1, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 1, 0, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 1, 0, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 1, 1, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 0, 1, 1, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 1, 0, 0, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 1, 0, 0, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 1, 0, 1, 0])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 1, 0, 1, 1])  
 ReservoirState(Int64[], [0, 0, 0, 0, 0, 1, 1, 0, 0])  
 ⋮                                                     
 ReservoirState([9, 8, 7], [1, 1, 1, 1, 1, 0, 1, 0, 0])
 ReservoirState([9, 8, 7], [1, 1, 1, 1, 1, 0, 1, 0, 1])
 ReservoirState([9, 8, 7], [1, 1, 1, 1, 1, 0, 1, 1, 0])
 Reservo

In [6]:
# Returns number of states
function POMDPs.n_states(pomdp::ReservoirPOMDP)
    return length(states(pomdp))
end

In [7]:
# Action space definition
println("Generating action space.")

# List of possible actions
action_space = []

# Iterate through all possible actions (i.e. pick a grid)
for x = 1:pomdp.size_grid
    push!(action_space, x)
end

println("Action space generated.")
println("Length of action_space: ", length(action_space))

POMDPs.actions(pomdp::ReservoirPOMDP) = action_space

# Test
POMDPs.actions(pomdp)

Generating action space.
Action space generated.
Length of action_space: 9


9-element Array{Any,1}:
 1
 2
 3
 4
 5
 6
 7
 8
 9

In [8]:
# Returns number of actions
function POMDPs.n_actions(pomdp::ReservoirPOMDP)
    return length(actions(pomdp))
end

In [9]:
# Observation space definition

# List of possible observations
obs_space = []

# List of possible observation values for *each grid*
ind_obs = []
for o1 = 0:1
    push!(ind_obs, o1)
end

# Genearate all possible observations
for i = 1:pomdp.size_grid
    for g1 = 1:length(ind_obs), g2 = 1:length(ind_obs), g3 = 1:length(ind_obs), g4 = 1:length(ind_obs),
        g5 = 1:length(ind_obs), g6 = 1:length(ind_obs), g7 = 1:length(ind_obs), g8 = 1:length(ind_obs),
        g9 = 1:length(ind_obs)
        o = [ind_obs[g1], ind_obs[g2], ind_obs[g3], ind_obs[g4],
             ind_obs[g5], ind_obs[g6], ind_obs[g7], ind_obs[g8], ind_obs[g9]]
        push!(obs_space, o)
    end
end

println("Observation space generated.")
println("Length of obs_space: ", length(obs_space))

POMDPs.observations(pomdp::ReservoirPOMDP) = obs_space

# Test
POMDPs.observations(pomdp)

Observation space generated.
Length of obs_space: 4608


4608-element Array{Any,1}:
 [0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 1]
 [0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 0, 1]
 [0, 0, 0, 0, 0, 0, 1, 1, 0]
 [0, 0, 0, 0, 0, 0, 1, 1, 1]
 [0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 1]
 [0, 0, 0, 0, 0, 1, 0, 1, 0]
 [0, 0, 0, 0, 0, 1, 0, 1, 1]
 [0, 0, 0, 0, 0, 1, 1, 0, 0]
 ⋮                          
 [1, 1, 1, 1, 1, 0, 1, 0, 0]
 [1, 1, 1, 1, 1, 0, 1, 0, 1]
 [1, 1, 1, 1, 1, 0, 1, 1, 0]
 [1, 1, 1, 1, 1, 0, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 0, 0, 0]
 [1, 1, 1, 1, 1, 1, 0, 0, 1]
 [1, 1, 1, 1, 1, 1, 0, 1, 0]
 [1, 1, 1, 1, 1, 1, 0, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 0, 0]
 [1, 1, 1, 1, 1, 1, 1, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
# Returns number of observations
function POMDPs.n_observations(pomdp::ReservoirPOMDP)
    return length(observations(pomdp))
end

## Reward Function

In [11]:
# Immediate reward, R(s,a)
# In our model R(s,a) is invariant on sp
function POMDPs.reward(pomdp::ReservoirPOMDP, s::ReservoirState, a::Int64)
    if (a in s.drilled)
        # If previously drilled: Get no oil & apply penalty
        return pomdp.penalty
    else
        # Otherwise: Take oil
        return s.grid[a]
    end
end

POMDPs.reward(pomdp::ReservoirPOMDP, s::ReservoirState, a::Int64, sp::ReservoirState) = reward(pomdp, s, a);

## Transition/Observation/Initial-State Distribution Definition

In [12]:
# Transition distribution
struct TransitionDistribution
    s::ReservoirState
    a::Int64
end
function POMDPs.transition(pomdp::ReservoirPOMDP, s::ReservoirState, a::Int64)
    return TransitionDistribution(s, a)
end

# Calculates T(sp | s,a)
# Note that in our model the transition is deterministic
function POMDPs.pdf(d::TransitionDistribution, sp::ReservoirState)
    history = copy(d.s.drilled)
    push!(history, a)
    if ((history == sp.drilled) && (s.grid == sp.grid))
        # T(sp | s,a)=1 iff s.drilled + action == sp.drilled
        return 1.0
    else
        # T(sp | s,a)=0 otherwise
        return 0.0
    end
end

# Randomly sample one transition, given an <s,a> pair
function POMDPs.rand(rng::AbstractRNG, d::TransitionDistribution)
    new_drilled = copy(d.s.drilled)
    push!(new_drilled, d.a)
    return ReservoirState(new_drilled, copy(d.s.grid))
end

In [13]:
# Observation distribution for rock
struct RockObsDistribution
    s::ReservoirState
    p_rock::Float64
end
function POMDPs.observation(pomdp::ReservoirPOMDP, s::ReservoirState, a::Int64, sp::ReservoirState)
    return RockObsDistribution(sp, pomdp.p_rock)
end

# Calculates P(o | s)
function POMDPs.pdf(d::RockObsDistribution, o::Array{Int64})
#   P(o|s) = Product[for each grid](P( rock == o[this_grid] | oil == s[this_grid] ))
    prod = 1.0
    for i = 1:length(o)
        # Oil:  d.s.grid[i]
        # Rock: o[i]
        if (o[i] == d.s.grid[i])
            # P(o[i]==1 | s[i]==1) == P(o[i]==0 | s[i]==0) = d.p_rock
            prod = prod * d.p_rock
        else
            # P(o[i]==1 | s[i]==0) == P(o[i]==0 | s[i]==1) = (1 - d.p_rock)
            prod = prod * (1 - d.p_rock)
        end
    end
    return prod
end;

# Randomly sample one observation, given a true hidden state
function POMDPs.rand(rng::AbstractRNG, d::RockObsDistribution)
    rng = MersenneTwister(19)
    o = []
    for i = 1:9
        if Base.rand(rng, Float64) < d.p_rock
            push!(o, d.s.grid[i])
        else
            push!(o, 1-d.s.grid[i])
        end
    end
    return o
end

In [14]:
# Initial state distribution
struct InitialStateDistribution
    p_rock::Float64
end
function POMDPs.initial_state_distribution(pomdp::ReservoirPOMDP)
    return InitialStateDistribution(pomdp.p_rock)
end

# Randomly generates an initial state
# drilled = empty
# grid = Uniformly random 0/1 oil value for each grid
function POMDPs.rand(rng::AbstractRNG, d::InitialStateDistribution)
    rng = MersenneTwister(19)
    grid = []
    for i = 1:9
        if Base.rand(rng, Float64) < 0.5
            push!(grid, 0)
        else
            push!(grid, 1)
        end
    end
    return ReservoirState([], grid)
end

## Sample Instance Creation

In [ ]:
# Create an instance
pomdp = ReservoirPOMDP()